In [21]:


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [22]:


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
#set index to id column
train.set_index('id', inplace=True)
test.set_index('id', inplace=True)



In [23]:
train.tail()

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
id,,,,,,,,,,,,
54268,BMW,X6 xDrive50i,2017,29000,Gasoline,445.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Brown,None reported,Yes,29000
54269,Audi,A4 2.0T Premium,2015,94634,E85 Flex Fuel,220.0HP 2.0L 4 Cylinder Engine Flex Fuel Capab...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,6500
54270,Porsche,Cayenne S,2013,40989,Gasoline,420.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,White,Black,At least 1 accident or damage reported,Yes,18950
54271,Porsche,911 Carrera 4 GTS,2023,1518,Gasoline,4.0L H6 24V GDI DOHC,8-Speed Automatic with Auto-Shift,Beige,Brown,None reported,Yes,194965
54272,Audi,A5 Sportback S line Premium Plus,2021,35000,Hybrid,261.0HP 2.0L 4 Cylinder Engine Gasoline/Mild E...,8-Speed A/T,Black,Black,None reported,Yes,37499


In [24]:
def age(df):
    df['age'] =2024 - df['model_year']
    df.drop('model_year', axis=1, inplace=True)
    df.drop('clean_title', axis=1, inplace=True)

age(train)
age(test)



In [25]:
train['fuel_type'].replace('–', 'Electric', inplace = True)
test['fuel_type'].replace('–', 'Electric', inplace = True)
train['fuel_type'].replace('not supported', 'Electric', inplace = True)
test['fuel_type'].replace('not supported', 'Electric', inplace = True)


In [26]:


print(test['fuel_type'].value_counts())
print(train['fuel_type'].value_counts())



fuel_type
Gasoline          33033
Hybrid             1112
E85 Flex Fuel      1018
Diesel              671
Electric            201
Plug-In Hybrid      148
Name: count, dtype: int64
fuel_type
Gasoline          49439
Hybrid             1766
E85 Flex Fuel      1479
Diesel             1109
Electric            298
Plug-In Hybrid      182
Name: count, dtype: int64


In [27]:
import re
# Define function to extract power
def extract_power(engine_type):
    match = re.search(r'(\d+\.\d+)HP|\d+\.\d+', engine_type)
    return match.group(1) if match else np.nan

# Define function to extract displacement
def extract_displacement(engine_type):
    match = re.search(r'(\d+\.\d+)L', engine_type)
    return match.group(1) if match else np.nan

# Define function to extract number of cylinders
def extract_cylinders(engine_type):
    match = re.search(r'(\d+)\s*Cylinders?', engine_type, re.IGNORECASE)
    return match.group(1) if match else np.nan

train['power'] = train['engine'].apply(extract_power)
test['power'] = test['engine'].apply(extract_power)
train['displacement'] = train['engine'].apply(extract_displacement)
test['displacement'] = test['engine'].apply(extract_displacement)
train['cylinders'] = train['engine'].apply(extract_cylinders)
test['cylinders'] = test['engine'].apply(extract_cylinders)

In [28]:
train['power'] = pd.to_numeric(train['power'], errors = 'coerce')
test['power'] = pd.to_numeric(test['power'], errors = 'coerce')


In [29]:


train['power'].fillna(train['power'].mean(), inplace = True)
test['power'].fillna(test['power'].mean(), inplace = True)



In [30]:
train['displacement'] = pd.to_numeric(train['displacement'], errors = 'coerce')
test['displacement'] = pd.to_numeric(test['displacement'], errors = 'coerce')

train['displacement'].fillna(train['displacement'].mean(), inplace = True)
test['displacement'].fillna(test['displacement'].mean(), inplace = True)

train['cylinders'] = pd.to_numeric(train['cylinders'], errors = 'coerce')
test['cylinders'] = pd.to_numeric(test['cylinders'], errors = 'coerce')

train['cylinders'].fillna(train['cylinders'].mean(), inplace = True)
test['cylinders'].fillna(test['cylinders'].mean(), inplace = True)


In [31]:


train.drop('engine', axis=1, inplace=True)
test.drop('engine', axis=1, inplace=True)



In [32]:


train.drop('model', axis=1, inplace=True)
test.drop('model', axis=1, inplace=True)
train.drop('ext_col', axis=1, inplace=True)
test.drop('ext_col', axis=1, inplace=True)
train.drop('int_col', axis=1, inplace=True)
test.drop('int_col', axis=1, inplace=True)



In [33]:


# For the 'train' DataFrame
train['accident'] = train['accident'].apply(lambda x: 1 if x == 'None reported' else 0)

# For the 'test' DataFrame
test['accident'] = test['accident'].apply(lambda x: 1 if x == 'None reported' else 0)



In [34]:
import re

# Define function to extract transmission type
def extract_transmission(engine_type):
    # Define a pattern that matches the desired terms
    pattern = r'\b(A/T|Automatic|CVT|w/Dual Shift|M/T)\b'
    match = re.search(pattern, engine_type)
    return match.group(1) if match else None

# Apply the extract_transmission function to the 'engine_type' column
train['new_trans'] = train['transmission'].apply(extract_transmission)
test['new_trans'] = test['transmission'].apply(extract_transmission)



In [35]:


test.drop(['fuel_type'], axis=1, inplace=True)
train.drop(['fuel_type'], axis=1, inplace=True)


In [36]:
#lable encode
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['brand'] = le.fit_transform(train['brand'])
test['brand'] = le.fit_transform(test['brand'])
train['transmission'] = le.fit_transform(train['transmission'])
test['transmission'] = le.fit_transform(test['transmission'])
train['new_trans'] = le.fit_transform(train['new_trans'])
test['new_trans'] = le.fit_transform(test['new_trans'])

In [37]:
train.tail()

,brand,milage,transmission,accident,price,age,power,displacement,cylinders,new_trans
id,,,,,,,,,,
54268,4,29000,27,1,29000,7,445.000000,4.4,8.000000,0
54269,3,94634,14,0,6500,9,220.000000,2.0,4.000000,0
54270,41,40989,43,0,18950,11,420.000000,3.6,6.000000,4
54271,41,1518,29,1,194965,1,331.698323,4.0,6.282347,1
54272,3,35000,27,1,37499,3,261.000000,2.0,4.000000,0


In [38]:
from sklearn.model_selection import train_test_split
X = train.drop(columns=['price'])
y = train['price']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# from sklearn.preprocessing import Normalizer

# scaler = Normalizer()
# X_train_scaled = scaler.fit_transform(X_train)
# X_val_scaled = scaler.transform(X_val)


In [39]:
#display the scaled data
#pd.DataFrame(X_train_scaled, columns=X_train.columns).head()

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

## Define the neural network model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1)
])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), batch_size=32, callbacks=[early_stopping, reduce_lr])

Epoch 1/100


/home/tanmay/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1357/1357 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 5877459968.0000 - mae: 29584.5586 - val_loss: 2464645632.0000 - val_mae: 18617.1230 - learning_rate: 0.0010
Epoch 2/100
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 4762933248.0000 - mae: 20456.2969 - val_loss: 2517216768.0000 - val_mae: 18992.7129 - learning_rate: 0.0010
Epoch 3/100
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 5115459072.0000 - mae: 20331.8398 - val_loss: 2670816768.0000 - val_mae: 19913.9434 - learning_rate: 0.0010
Epoch 4/100
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 3606514432.0000 - mae: 19027.3945 - val_loss: 2603310848.0000 - val_mae: 18905.9199 - learning_rate: 0.0010
Epoch 5/100
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 5736340480.0000 - mae: 20393.1113 - val_loss: 2439234816.0000 - val_mae: 16911.6367 - learning_rate: 0.0010
Epoch 6/100
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 6316354560.0000 - mae: 20133.8398 - val_loss: 2438755584.0000 - val_mae: 17037.7480 - learning

In [41]:
val_loss, val_mae = model.evaluate(X_val, y_val)
print(f'Validation MAE: {val_mae}')
X_test= test

340/340 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 2711514624.0000 - mae: 17158.3848
Validation MAE: 17209.4375


In [42]:
predictions = model.predict(X_test)

1131/1131 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [43]:
print(len(predictions))

36183


In [44]:
test['price'] = predictions
test['id'] = test.index
test = test[['id', 'price']]
test.to_csv('submission.csv', index=False)